In [37]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pathlib import Path
import time

In [38]:
# Selenium options required to create a 'headless' browser
options = Options()
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.37")

driver = webdriver.Chrome(options=options)

In [39]:
# Load ridership page from BMRCL website
driver.get("https://english.bmrc.co.in/ridership/")

# Click on Kannada toggle button to load English results
time.sleep(10)  # Allow extra time for JavaScript to load translated data
result = driver.find_element(By.CLASS_NAME, "link.top-navcustom-text").click()

In [40]:
# Results are published with a lag of about one day. 
# So get the date on the page rather than date.today()
record_date = driver.find_element(By.TAG_NAME, "h3").text
record_date

'Passenger Flow as on 03-12-2024'

In [41]:
# Initialize dict to store ridership data
day_record = {}
day_record['Record Date'] = [record_date.split()[-1]]  # Extracting date part
day_record

{'Record Date': ['03-12-2024']}

In [42]:
# Parse html for remaining data points and store in pandas dataframe
data_points = driver.find_elements(By.CLASS_NAME, "features-card.achivement-area.bg-color")

for l1 in data_points:
    for l2 in l1.text.split('\n'):
        data = l2.split(': ')
        day_record[data[0]] = [int(data[1])]

driver.quit()

day_record = pd.DataFrame(day_record)
day_record

,Record Date,Total Smart Cards,Stored Value Card,One Day Pass,Three Day Pass,Five Day Pass,Tokens,Total NCMC,Group Ticket,Total QR,QR NammaMetro,QR WhatsApp,QR Paytm
0,03-12-2024,461996,461278,429,95,194,198932,12319,325,155053,51715,65741,37597


In [43]:
# Store data in csv file - create file if necessary
filename = "NammaMetro_Ridership_Dataset.csv"  
filePath = Path(filename)

if filePath.exists() and filePath.is_file():
    day_record.to_csv(filename, mode='a', header=False)
    print('Appended '+filename)
else:
    day_record.to_csv(filename, mode='w', header=True)
    print('Created '+filename)

Appended NammaMetro_Ridership_Dataset.csv


In [45]:
# Optimize dataset by removing duplicates and rewrite to file
df = pd.read_csv(filename, index_col=0).drop_duplicates(keep='last', ignore_index=True)
df[df.loc[:, 'Total Smart Cards':].columns] = df[df.loc[:, 'Total Smart Cards':].columns].astype('Int64')
df.to_csv(filename, mode='w', header=True)
df.tail()


,Record Date,Total Smart Cards,Stored Value Card,One Day Pass,Three Day Pass,Five Day Pass,Tokens,Total NCMC,Group Ticket,Total QR,QR NammaMetro,QR WhatsApp,QR Paytm
28,29-11-2024,459914,458983,644,77,210,205777,11220,780,181937,50383,97180,34374
29,30-11-2024,391894,390520,1180,142,52,234756,8428,1045,178402,47520,97787,33095
30,01-12-2024,154334,153369,816,110,39,220404,4122,600,196242,52072,107955,36215
31,02-12-2024,409724,409132,340,90,162,187408,10402,872,177203,50580,93367,33256
32,03-12-2024,461996,461278,429,95,194,198932,12319,325,155053,51715,65741,37597
